In [2]:
from sympy import *
from sympy.physics.mechanics import dynamicsymbols, mechanics_printing
mechanics_printing()
import modern_robotics as mr
import numpy as np

### Functions & variables from former tasks

#### Functions

In [4]:
def skew(v):
    return Matrix([[0, -v[2], v[1]],
                   [v[2], 0, -v[0]],
                   [-v[1], v[0], 0]])

def exp6(twist, theta):
    omega = skew(twist[:3])
    v = Matrix(twist[3:])
    T = eye(4)
    T[:3, :3] = exp3(twist[:3], theta)
    T[:3, 3] = (eye(3) * theta + (1 - cos(theta)) * omega +
                (theta - sin(theta)) * omega * omega) * v
    return T

def exp3(omega, theta):
    omega = skew(omega)
    R = eye(3) + sin(theta) * omega + (1 - cos(theta)) * omega * omega
    return R

def Ad(T):
    AdT = zeros(6)
    R = Matrix(T[:3, :3])
    AdT[:3, :3] = R
    AdT[3:, 3:] = R
    AdT[3:, :3] = skew(T[:3, 3]) * R
    return AdT

def Ad(T):
    AdT = zeros(6)
    R = Matrix(T[:3, :3])
    AdT[:3, :3] = R
    AdT[3:, 3:] = R
    AdT[3:, :3] = skew(T[:3, 3]) * R
    return AdT

def rotX(alfa_im1):
    Rx = eye(4)
    Rx[1,1] =    cos(alfa_im1)
    Rx[1,2] =   -sin(alfa_im1)
    Rx[2,1] =    sin(alfa_im1)
    Rx[2,2] =    cos(alfa_im1)
    return Rx

def rotZ(alfa_im1):
    Rz = eye(4)
    Rz[0,0] =    cos(alfa_im1)
    Rz[0,1] =   -sin(alfa_im1)
    Rz[1,0] =    sin(alfa_im1)
    Rz[1,1] =    cos(alfa_im1)
    return Rz

def transX(a_im1):
    trA = eye(4)
    trA[0,3] =  a_im1
    return trA

def transZ(d_i):
    trA = eye(4)
    trA[2,3] =  d_i
    return trA

def Js_maker(Slist, theta_list):
    n_joints = Slist.shape[1]
    Js = zeros(6, n_joints)

    for i in range(n_joints-1, -1, -1):
        if i==0: # legger til denne for å få Js[0] = S_sb[0], står i boka
            Js[:,i] = Slist[:,i]
                    
        else:
            T = exp6(Slist[:,i-1], theta_list[i-1])

            for j in range( i-2, -1, -1):
                T = exp6(Slist[:,j], theta_list[j]) * T

        Js[:,i] = Ad(T) * Slist[:,i]
        

    Js.simplify()
    return Js

def Jb_maker(Blist, theta_list):
    n_joints = Blist.shape[1] - 1
    Jb = zeros(6, 6)
    print(n_joints)
    
    Jb[:,n_joints] = Blist[:,n_joints] # Jb[n] = B[n]
    
    for i in range(n_joints-1, -1, -1):
        T = eye(4)#exp6(Blist[:,i], -theta_list[i])
        print("i",i)
        for j in range( i+1, n_joints+1):
            T = exp6(Blist[:,j], -theta_list[j]) * T
            print("j",j)
        Jb[:,i] = Ad(T) * Blist[:,i]
        print("\n")
    
    return Jb





#### Variables

In [22]:
th1, th2, th3, th4, th5, th6 = dynamicsymbols('theta_1, theta_2, theta_3, theta_4, theta_5, theta_6')

S_bp = Matrix([[-1, 0, 0, 0, 0, 0], [0, 1, 1, 0, 1, 0], [0, 0, 0, 1, 0, 1], [0, -875, -420, 0, 0, 0], [-900, 0, 0, 0, 0, 0], [0, -35, -35, 0, 0, 0]])
S_bp = Matrix([[-1, 0, 0, 0, 0, 0], [0, -1, -1, 0, -1, 0], [0, 0, 0, -1, 0, 1], [0, -875, -420, 0, 0, 0], [900, 0, 0, 0, 0, 0], [0, 35, 35, 0, 0, 0]])
S_bp

⎡-1    0     0    0   0   0⎤
⎢                          ⎥
⎢ 0    -1    -1   0   -1  0⎥
⎢                          ⎥
⎢ 0    0     0    -1  0   1⎥
⎢                          ⎥
⎢ 0   -875  -420  0   0   0⎥
⎢                          ⎥
⎢900   0     0    0   0   0⎥
⎢                          ⎥
⎣ 0    35    35   0   0   0⎦

## Le task

In [23]:
Jb = Jb_maker(S_bp, [th1,th2,th3,th4,th5,th6])


5
i 4
j 5


i 3
j 4
j 5


i 2
j 3
j 4
j 5


i 1
j 2
j 3
j 4
j 5


i 0
j 1
j 2
j 3
j 4
j 5




In [24]:
Jb

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎣-((((-35⋅sin(θ₂) + 875⋅cos(θ₂) - 875)⋅sin(θ₃) + (875⋅sin(θ₂) + 35⋅cos(θ₂) - 3

                                                                              
                                                   

In [25]:
Jb_12 = zeros(3,3) # Arm
Jb_21 = zeros(3,3) # Wrist

Jb_12 = Jb[0:3,3:6]
Jb_21 = Jb[3:6,0:3]

Jb_21


⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎣-((((-35⋅sin(θ₂) + 875⋅cos(θ₂) - 875)⋅sin(θ₃) + (875⋅sin(θ₂) + 35⋅cos(θ₂) - 3

            -(-(((-35⋅sin(θ₂) + 875⋅cos(θ₂) - 875)⋅sin(θ₃) + (875⋅sin(θ₂) + 35
                                                                              
            -(-(((-35⋅sin(θ₂) + 875⋅cos(θ₂) - 875)⋅sin(θ₃) + (875⋅sin(θ₂) + 35
                                                                              
5)⋅cos(θ₃) + 420⋅sin(θ₃) + 35⋅cos(θ₃) - 35)⋅cos(θ₄)⋅cos(θ₅) + ((-35⋅sin(θ₂) + 

⋅cos(θ₂) - 35)⋅cos(θ₃) + 420⋅sin(θ₃) + 35⋅cos(θ₃) - 35)⋅cos(θ₄)⋅cos(θ₅) + ((-3
                                                                              
⋅cos(θ₂) - 35)⋅cos(θ₃) + 420⋅sin(θ₃) + 35⋅cos(θ₃) 

In [26]:
det_Jb_12 = Jb_12.det()
det_Jb_12

           2                  2    
sin(θ₅)⋅sin (θ₆) + sin(θ₅)⋅cos (θ₆)

In [27]:
det_Jb_21 = Jb_21.det()

In [35]:
a = det_Jb_21.subs(th4,0)
b = a.subs(th5, 0)
c = b.subs(th6, 0)
d = c.subs(th1, 0)

d.simplify()


                                                79704625⋅sin(θ₂ + 2⋅θ₃)   8081
43475250⋅sin(θ₂ - θ₃) - 43475250⋅sin(θ₂ + θ₃) - ─────────────────────── + ────
                                                           2                  

9375⋅sin(θ₂)                     7245875⋅cos(θ₂ - θ₃)   7245875⋅cos(θ₂ + θ₃)  
──────────── - 4777500⋅sin(θ₃) + ──────────────────── + ──────────────────── +
   2                                      2                      2            

                                        
 6688500⋅cos(θ₂ + 2⋅θ₃) + 398125⋅cos(θ₃)
                                        

In [36]:
# s = solve(d, th2, th3)

KeyboardInterrupt: 

In [113]:
solutionArm = solve(det_Jb_12,th1,th2,th3,th4,th4,th5,th6)
solutionArm

[{θ₅: 0}, {θ₅: π}]